
## 러스트에서 수명 매개변수(lifetime parameter)

- 모든 참조에는 해당 참조가 유효한 범위인 수명이 있다는 것입니다. 
- 대부분의 경우 수명은 대부분의 경우 유형이 유추되는 것처럼 암시적이고 유추됩니다. 
- 여러 유형이 가능한 경우에만 유형에 주석을 달아야 합니다. 비슷한 방식으로 참조의 수명이 몇 가지 다른 방식으로 관련될 수 있는 경우 수명에 주석을 달아야 합니다. 
- Rust는 런타임에 사용되는 실제 참조가 확실히 유효하도록 일반 수명 매개변수를 사용하여 관계에 주석을 달도록 요구합니다.

### 명시적 수명 요구할 경우 
- 참조의 수명을 명시적으로 지정하기 위해 사용됩니다. 
- 수명 매개변수는 함수나 구조체의 정의에서 'a와 같은 형식으로 표시됩니다.

## 수명을 점검하는 이유  

- 컴파일러는 모든 변수의 lifetime을 이해하며 참조가 절대로 그 owner보다 더 오래 존재하지 못하도록 검증을 시도합니다.

- 함수에서는 어떤 매개변수와 리턴 값이 서로 같은 lifetime을 공유하는지 식별할 수 있도록 심볼로 표시하여 명시적으로 생명주기를 지정할 수 있습니다.

## 1. 참조 수명 매개변수 정의  : 함수

## 수명매개변수를 지정하지 않을 때 컴파일러가 필요하가고 지정한다. 

- 두 개의 참조를 받는데 이중에 하나가 반환이 되는데 수명이 불명확하다 

In [21]:
fn longest(x: &str, y: &str) -> &str {
    if x.len() > y.len() {
        x
    } else {
        y
    }
}

fn main() {
    let string1 = String::from("abcd");
    let string2 = "xyz";

    let result = longest(string1.as_str(), string2);
    println!("The longest string is {}", result);
}

main();

Error: missing lifetime specifier

## 수명매개변수를 지정하기

- 동일한 수명을 알려주기 위해 수명 매개변수를 지정

In [22]:
fn longest<'a>(x: &'a str, y: &'a str) -> &'a str {
    if x.len() > y.len() {
        x
    } else {
        y
    }
}

fn main() {
    let s1 = "hello";
    let s2 = "world";
    let result = longest(s1, s2);
    println!("The longest string is: {}", result);
}
main();

The longest string is: world


###  두개이상이 수명매개변수를 사용할 경우 반환을 하나로 하는 경우 오류

- 그래서 이럴 경우는 2개의 수명매개변수를 가지고 반환처리 

In [7]:
fn longest3<'a, 'b>(x: &'a str, y: &'b str) -> (&'a str, &'b str) {
    if x.len() > y.len() {
        (x,y)
    } else {
        (x,y)
    }
}

fn main() {
    let x = "Hello, world!";
    let y = &x;
    
    let longest_string = longest3(&x, y);
    
    println!("{:?}", longest_string);

}
main();

("Hello, world!", "Hello, world!")


## 함수 매개변수가 구조체일 경우 수명으로 처리하기 

In [31]:
struct Foo {
    x: i32,
}

                                                  // 매개변수 foo와 리턴 값은 동일한 lifetime을 공유함
fn do_something<'a>(foo: &'a Foo) -> &'a i32 {
    return &foo.x;
}

fn main() {
    let mut foo = Foo { x: 42 };
    let x = &mut foo.x;
    *x = 13;
                                                  // x가 여기서 drop 되어, non-mutable 참조를 생성할 수 있음
    let y = do_something(&foo);
    println!("{}", y);
}                                                  // y는 여기서 drop 됨
                                                   // foo는 여기서 drop 됨

main();

13


In [24]:
struct Foo3 {
    x: i32,
}

// foo_b와 리턴 값은 동일한 lifetime을 공유함
// foo_a는 무관한 lifetime을 가짐
fn do_something<'a, 'b>(foo_a: &'a Foo3, foo_b: &'b Foo3) -> &'b i32 {
    println!("{}", foo_a.x);
    println!("{}", foo_b.x);
    return &foo_b.x;
}

fn main() {
    let foo_a = Foo3 { x: 42 };
    let foo_b = Foo3 { x: 12 };
    let x = do_something(&foo_a, &foo_b);
    // 여기 이후에는 foo_b의 lifetime만 존재하므로 foo_a만 drop 됨
    println!("{}", x);
    // 여기서 x가 drop 됨
    // 여기서 foo_b가 drop 됨
}

main();

42
12
12


## 2. 참조 수명 매개변수 정의  : 구조체

## 데이터 자료형의 생명주기

- 함수와 마찬가지로, 데이터 자료형의 구성원들도 lifetime 지정자로 지정할 수 있습니다.

- Rust는 참조가 품고 있는 데이터 구조가 참조가 가리키는 owner보다 절대 오래 살아남지 못하도록 검증합니다.

- 아무 것도 아닌 것을 가리키는 참조를 들고 다니는 struct란 있을 수 없습니다!

In [27]:
struct Foo<'a> {
    i:&'a i32
}

fn main() {
    let x = 42;
    {
        let foo = Foo {
            i: &x
        };
    };
    println!("{}",foo.i);
}

main();

Error: cannot find value `foo` in this scope

In [28]:
fn main() {
    let x = 42;
    
    let foo = Foo {
            i: &x
        };

    println!("{}",foo.i);
}

main();

42


In [2]:
struct S<'a> {
    x : &'a i32,
    y : &'a i32,
}

In [4]:
fn main() {
    let x = 10;                            // x의 시작 
    let r ;

    {
        let y = 20;                        // y의 시작
        {
            let s = S {x:&x, y:&y};
            r = s.x;                       // r의 시작 
        }
    }

    println!(" {} ", r);
}

main();

Error: `y` does not live long enough

## 변수를 지정하는 위치를 변경 

In [5]:
fn main() {
    let x = 10;                            // x의 시작 
    let y = 20;                            // y의 시작
    let r ;

    {
        {
            let s = S {x:&x, y:&y};
            r = s.x;                       // r의 시작 
        }
    }

    println!(" {} ", r);
}

main();

 10 


## 구조체의 수명매개변수를 별도로 지정

In [6]:
struct S2<'a,'b> {
    x : &'a i32,
    y : &'b i32,
}

In [7]:
fn main() {
    let x = 10;                            // x의 시작 
    let r ;

    {
        let y = 20;                        // y의 시작
        {
            let s = S2 {x:&x, y:&y};
            r = s.x;                       // r의 시작 
        }
    }

    println!(" {} ", r);
}

main();

 10 


## 3. 정적인 생명주기 

### 정적인 생명주기
- static 변수는 컴파일 타임에 생성되어 프로그램의 시작부터 끝까지 존재하는 메모리 리소스입니다. 이들은 명시적으로 자료형을 지정해 주어야 합니다.

- static lifetime은 프로그램이 끝날 때까지 무한정 유지되는 메모리 리소스입니다. 이 정의에 따르면, 어떤 static lifetime의 리소스는 런타임에 생성될 수도 있다는 점 알아두세요.

- static lifetime을 가진 리소스는 'static이라는 특별한 lifetime 지정자를 갖습니다.

- 'static한 리소스는 절대 drop 되지 않습니다.

- 만약 static lifetime을 가진 리소스가 참조를 포함하는 경우, 그들도 모두 'static이어야 합니다 (그 이하의 것들은 충분히 오래 살아남지 못합니다).

In [25]:
static PI: f64 = 3.1415;

fn main() {
    // static 변수는 함수 scope 안에도 넣을 수 있습니다
    static mut SECRET: &'static str = "swordfish";

    // string 값들은 'static lifetime을 갖습니다
    let msg: &'static str = "Hello World!";
    let p: &'static f64 = &PI;
    println!("{} {}", msg, p);

    // 일부 규칙은 깰 수 있으나, 반드시 명시적으로 해야 합니다
    unsafe {
        // SECRET에 string 값을 설정할 수 있는데, 이 값 역시 'static이기 때문입니다
        SECRET = "abracadabra";
        println!("{}", SECRET);
    }
}

main();

Hello World! 3.1415
abracadabra


## 4. 수명매개변수와 제너릭을 같이 처리하기

In [12]:
fn first_element<'a, T>(vec: &'a Vec<T>) -> Option<&'a T> {
    vec.first()
}

fn main() {
    let vec = vec![1, 2, 3, 4, 5];
    let result = first_element(&vec);
    
    match result {
        Some(value) => println!("The first element is: {}", value),
        None => println!("The vector is empty."),
    }
}

main();

The first element is: 1
